In [13]:
import pandas as pd
import numpy as np
import os
import openpyxl
import psycopg2

In [14]:
df = pd.read_excel("vendor_payments.xlsx")
df.head()

,payment_id,payment_number,bill_numbers,vendor_id,vendor_name,payment_mode,description,date,reference_number,amount,...,is_paid_via_print_check,paid_through_account_name,check_id,check_number,check_status,offset_account_name,payment_type,currency_code,exchange_rate,branch_name
0,2620362000001382181,1875,KTZ10021,2620362000000669931,One Stop Logistics,Cash,NaN,2021-01-05,NaN,2500.0,...,False,Bank of America Transactions,NaN,NaN,NaN,Accounts Payable,bill_payment,USD,5.868200,Jetstream Africa
1,2620362000001382207,1876,18 December,2620362000000669967,Mohammed Tofic,Cash,NaN,2021-01-05,NaN,1000.0,...,False,MTN Mobile Money,NaN,NaN,NaN,Prepaid Expenses,bill_payment,GHS,1.000000,Jetstream Africa
2,2620362000001382239,1877,AA-0105,2620362000000669993,Baba Packaging,Cash,NaN,2021-01-05,NaN,720.0,...,False,MTN Mobile Money,NaN,NaN,NaN,Accounts Payable,bill_payment,GHS,1.000000,Jetstream Africa
3,2620362000001382261,1878,CNT# BEAU4725271,2620362000000670019,Michael Quartey Transport Service,Cash,NaN,2021-01-05,NaN,2600.0,...,False,MTN Mobile Money,NaN,NaN,NaN,Accounts Payable,bill_payment,GHS,1.000000,Jetstream Africa
4,2620362000001382283,1879,2119153389,2620362000000670055,Hapag Lloyd USD,Cash,NaN,2021-01-06,NaN,657.0,...,False,Bank of America Transactions,NaN,NaN,NaN,Accounts Payable,bill_payment,USD,5.863521,Jetstream Africa


In [15]:
df.columns = [x.lower().replace(" ","_").replace("%","") for x in df.columns]
df.columns

Index(['payment_id', 'payment_number', 'bill_numbers', 'vendor_id',
       'vendor_name', 'payment_mode', 'description', 'date',
       'reference_number', 'amount', 'bcy_amount', 'bcy_bank_charge',
       'unused_amount', 'bcy_unused_amount', 'paid_through_account_id',
       'is_paid_via_print_check', 'paid_through_account_name', 'check_id',
       'check_number', 'check_status', 'offset_account_name', 'payment_type',
       'currency_code', 'exchange_rate', 'branch_name'],
      dtype='object')

In [16]:
df.dtypes

payment_id                     int64
payment_number                 int64
bill_numbers                  object
vendor_id                      int64
vendor_name                   object
payment_mode                  object
description                   object
date                          object
reference_number              object
amount                       float64
bcy_amount                   float64
bcy_bank_charge                int64
unused_amount                float64
bcy_unused_amount            float64
paid_through_account_id        int64
is_paid_via_print_check         bool
paid_through_account_name     object
check_id                     float64
check_number                 float64
check_status                 float64
offset_account_name           object
payment_type                  object
currency_code                 object
exchange_rate                float64
branch_name                   object
dtype: object

In [17]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'varchar',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'varchar',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [18]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'payment_id varchar, payment_number varchar, bill_numbers varchar, vendor_id varchar, vendor_name varchar, payment_mode varchar, description varchar, date varchar, reference_number varchar, amount float, bcy_amount float, bcy_bank_charge varchar, unused_amount float, bcy_unused_amount float, paid_through_account_id varchar, is_paid_via_print_check varchar, paid_through_account_name varchar, check_id float, check_number float, check_status float, offset_account_name varchar, payment_type varchar, currency_code varchar, exchange_rate float, branch_name varchar'

In [19]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [20]:
cursor.execute("drop table if exists vendor_payments;")

In [21]:
cursor.execute("create table vendor_payments \
    (payment_id varchar, payment_number varchar, bill_numbers varchar, vendor_id varchar, vendor_name varchar, payment_mode varchar, description varchar, date date, reference_number varchar, amount float, bcy_amount float, bcy_bank_charge varchar, unused_amount float, bcy_unused_amount float, paid_through_account_id varchar, is_paid_via_print_check varchar, paid_through_account_name varchar, check_id float, check_number float, check_status float, offset_account_name varchar, payment_type varchar, currency_code varchar, exchange_rate float, branch_name varchar)")

In [22]:
#insert values to tables
#save df to csv
df.to_csv('vendor_payments', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("vendor_payments")
print('file opened in memory')

file opened in memory


In [23]:
SQL_STATEMENT = """
COPY vendor_payments FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [24]:
cursor.execute("grant select on table vendor_payments to public")
conn.commit()

cursor.close()
print('table vendor_payments imported successfully')

table vendor_payments imported successfully
